# Optimizing Piecewise Linear Policies in MILP

### Imports

In [1]:
from gurobipy import *
import numpy as np
import itertools

## PwL policies in MILP

We restrict the class of parametric policies $\pi_{\theta} \colon \mathcal{S} \rightarrow \mathcal{A}$ to be piecewise linear over an adaptive rectlinear domain:

$$
\pi_{\theta}(\mathbf{s}) = \begin{cases}
\mathbf{w}_1^{\text{T}} \cdot \phi(\mathbf{s})&:&\mathbf{l}_1 \leq \mathbf{s} \leq \mathbf{u}_1 \\
\mathbf{w}_2^{\text{T}} \cdot \phi(\mathbf{s})&:&\mathbf{l}_2 \leq \mathbf{s} \leq \mathbf{u}_2 \\
 \cdots \\
\mathbf{w}_K^{\text{T}} \cdot \phi(\mathbf{s})&:&\mathbf{l}_K \leq \mathbf{s} \leq \mathbf{u}_K \\
\end{cases}
$$

where $\mathcal{P}(\mathcal{S}) = \{ (\mathbf{l}_k, \mathbf{u}_k) : i = k, \cdots, K\}$ forms a partition over the state space $\mathcal{S}$ and $\theta_k = (\mathbf{w}_k, (\mathbf{l}_k, \mathbf{u}_k))$ denotes the action parameters and bounds for the $k$-th partition case.

Let $\mathcal{I}\{l_{jk}\}$ and $\mathcal{I}\{u_{jk}\}$ be indicator variables such that:
$$
\begin{align}
\mathcal{I}\{l_{jk}\} = 1 & \leftrightarrow s_{j} \geq l_{jk} \\
\mathcal{I}\{u_{jk}\} = 1 & \leftrightarrow s_{j} \leq u_{jk}
\end{align}
$$ where $j$ is a vector dimensional index and $k$ is the case partition index.

We use [indicator constraints](http://www.gurobi.com/documentation/8.1/refman/py_model_addgenconstrindic.html) to encode $\mathcal{I}\{l_{jk}\}$ and $\mathcal{I}\{u_{jk}\}$ in MILP:
$$
\begin{align}
&\texttt{addGenConstrIndicator}(\mathcal{I}\{l_{jk}\}, \texttt{True}, s_{j} \geq l_{jk}) \\
&\texttt{addGenConstrIndicator}(\mathcal{I}\{l_{jk}\}, \texttt{False}, s_{j} \leq l_{jk}) \\
\end{align}
$$

$$
\begin{align}
&\texttt{addGenConstrIndicator}(\mathcal{I}\{u_{jk}\}, \texttt{True}, s_{j} \leq u_{jk}) \\
&\texttt{addGenConstrIndicator}(\mathcal{I}\{u_{jk}\}, \texttt{False}, s_{j} \geq u_{jk}) \\
\end{align}
$$

Let $\mathcal{I}\{l_k\}$ and $\mathcal{I}\{u_k\}$ be indicator variables such that:
$$
\begin{align}
\mathcal{I}\{l_{k}\} = 1 & \leftrightarrow \mathbf{s} \geq \mathbf{l}_{k} \\
\mathcal{I}\{u_{k}\} = 1 & \leftrightarrow \mathbf{s} \leq \mathbf{u}_{k}
\end{align}
$$ where vectors are compared element-wise.

We use [AND constraints](http://www.gurobi.com/documentation/8.1/refman/py_model_addgenconstrand.html) to encode $\mathcal{I}\{l_k\}$ and $\mathcal{I}\{u_k\}$ in MILP:
$$
\begin{align}
&\texttt{addGenConstrAnd}(\mathcal{I}\{l_k\}, [\mathcal{I}\{l_{1k}\}, \cdots, \mathcal{I}\{l_{Nk}\}])\\
&\texttt{addGenConstrAnd}(\mathcal{I}\{u_k\}, [\mathcal{I}\{u_{1k}\}, \cdots, \mathcal{I}\{u_{Nk}\}])
\end{align}
$$

Let $\mathcal{I}\{case_k\}$ be an indicator variable such that:
$$
\mathcal{I}\{case_k\} = 1 \leftrightarrow \mathbf{l}_k \leq \mathbf{s} \leq \mathbf{u}_k ~.
$$

We again use [AND constraints](http://www.gurobi.com/documentation/8.1/refman/py_model_addgenconstrand.html) to encode $\mathcal{I}\{case_k\}$ in MILP:
$$
\texttt{addGenConstrAnd}(\mathcal{I}\{case_k\}, [\mathcal{I}\{l_k\}, \mathcal{I}\{u_k\}])
$$

Finally, we guarantee the domain partition using [SOS1](http://www.gurobi.com/documentation/8.1/refman/py_model_addsos.html) and [OR constraints](http://www.gurobi.com/documentation/8.1/refman/py_model_addgenconstror.html):
$$
\begin{align}
\text{SOS1}(\mathcal{I}\{case_1\}, \cdots, \mathcal{I}\{case_K\})\\
\text{OR}(\mathcal{I}\{case_1\}, \cdots, \mathcal{I}\{case_K\}) & = 1 ~.
\end{align}
$$

### References
- [Gurobi API general constraints](http://www.gurobi.com/documentation/8.1/refman/constraints.html)

In [2]:
class PwLPolicy:

    def __init__(self, model, D, K, action_lb=None, action_ub=None, state_lb=None, state_ub=None):
        self.m = model
        self.D = D # dimensions
        self.K = K # number of cases in state partition
        self.action_lb = action_lb
        self.action_ub = action_ub
        self.state_lb = state_lb
        self.state_ub = state_ub

        self._build()

    def _build(self):
        self.w = self.m.addVars(self.K, self.D, self.D, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='w')
        self.b = self.m.addVars(self.K, self.D, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='b')
        self.l = self.m.addVars(self.K, self.D, lb=self.state_lb, ub=self.state_ub, name='l')
        self.u = self.m.addVars(self.K, self.D, lb=self.state_lb, ub=self.state_ub, name='u')

    def __call__(self, x, sample, t):
        action = self.m.addVars(self.D, lb=self.action_lb, ub=self.action_ub, name='a{}'.format(t))

        self.il = self.m.addVars(self.K, self.D, vtype=GRB.BINARY, name='il{}'.format(t))
        self.m.addConstrs((self.il[k, i] == True)  >> (self.l[k, i] <= x[i]) for k in range(self.K) for i in range(self.D))
        self.m.addConstrs((self.il[k, i] == False) >> (self.l[k, i] >= x[i]) for k in range(self.K) for i in range(self.D))

        self.iu = self.m.addVars(self.K, self.D, vtype=GRB.BINARY, name='iu{}'.format(t))
        self.m.addConstrs((self.iu[k, i] == True)  >> (self.u[k, i] >= x[i]) for k in range(self.K) for i in range(self.D))
        self.m.addConstrs((self.iu[k, i] == False) >> (self.u[k, i] <= x[i]) for k in range(self.K) for i in range(self.D))

        case = m.addVars(self.K, vtype=GRB.BINARY, name='case{}'.format(t))
        self.m.addConstrs((case[k] == and_(*self.il.select(k, '*'), *self.iu.select(k, '*')) for k in range(self.K)))

        self.m.addConstrs((case[k] == True) >> (action[i] == quicksum([self.w[k, i, j] * sample[j] for j in range(self.D)]) + self.b[k, i])
                          for k in range(self.K)
                          for i in range(self.D))

        self.m.addSOS(GRB.SOS_TYPE1, [*case.select('*')])
        partition = self.m.addVar(vtype=GRB.BINARY, name='partition{}'.format(t))
        self.m.addConstr(partition == or_(*case.select('*')))
        self.m.addConstr(partition == True)

        return action
    
    def __str__(self):
        pi_str = ''
        for k in range(self.K):
            pi_str += '-- ('
            for i in range(self.D):
                if i > 0:
                    pi_str += ', '
                for j in range(self.D):
                    if j > 0:
                        pi_str += ' + '
                    pi_str += '{:6.3f} * x{}'.format(self.w[k, i, j].x, j+1)
                pi_str += ' + {:6.3f}'.format(self.b[k, i].x)
            pi_str += ')'
            
            pi_str += ' if '
            for i in range(self.D):
                if i > 0:
                    pi_str += ' and '
                pi_str += 'x{} in [{:6.3f}, {:6.3f}]'.format(i+1, self.l[k, i].x, self.u[k, i].x)
            
            pi_str += '\n'
        
        return pi_str

## 1-step Decision-Making

### Optimizing 1-D Piecewise Constant Policy

**Problem specification**

Let $x \in [-1.0, 1.0]$ be a state variable and $a \in [-1.0, 1.0]$ be an action variable. We define $R(x, a)$ to be the following piecewise linear reward function:

$$
R(x, a) = \begin{cases}
+a, & x \leq 0 \\
-a, & x > 0 \\
\end{cases}.
$$

Let's consider a piecewise constant parameterized policy $\pi_{\theta}$ such that:
$$
\pi_{\theta}(x) = \begin{cases}
c_1, & l_1 \leq x \leq u_1 \\
c_2, & l_2 \leq x \leq u_2
\end{cases},
$$
where $\theta = \{ (c_i, l_i, u_i)~|~c_i, l_i, u_i \in \mathbb{R}, i=1,2 \}$ are the policy parameters.

Our objective is to maximize $R(x,a)$.

In [3]:
m = Model('PwL-Policy')

# generate states
N = 100
samples = [[x] for x in np.linspace(-0.90, 0.90, num=N)]
states = m.addVars(N, 1, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='x')
m.addConstrs((states[j, 0] == samples[j][0] for j in range(N)))

# encode PwL policy with rectlinear partition
K, D = 2, 1 
pi = PwLPolicy(m, D, K, action_lb=-1.0, action_ub=1.0, state_lb=-1.0, state_ub=1.0)
actions = []
for n in range(N):
    sample_n = samples[n]
    x_n = states.select(n, '*')
    a_n = pi(x_n, sample_n, n)
    actions.append(a_n)

# encode PwL cost function
for j in range(N):
    x_j, a_j = states.select(j, 0)[0], actions[j][0]
    r_j = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, obj=-1.0/N, name='c{}'.format(j)) # maximize

    ir = m.addVar(vtype=GRB.BINARY, name='ir_x{}'.format(j))
    m.addGenConstrIndicator(ir, True,  x_j <= 0.0)
    m.addGenConstrIndicator(ir, False, x_j >= 0.0)
    m.addGenConstrIndicator(ir, True,  r_j == a_j)
    m.addGenConstrIndicator(ir, False, r_j == -a_j)

# optimize policy parameters
m.optimize()

# report results
print('\n>> Policy:')
print(pi)

Academic license - for non-commercial use only
Optimize a model with 200 rows, 1108 columns and 200 nonzeros
Model has 100 SOS constraints
Model has 1700 general constraints
Variable types: 308 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-03, 1e+00]
Presolve added 1472 rows and 0 columns
Presolve removed 0 rows and 208 columns
Presolve time: 0.57s
Presolved: 1672 rows, 900 columns, 4716 nonzeros
Presolved model has 200 SOS constraint(s)
Variable types: 308 continuous, 592 integer (592 binary)
Found heuristic solution: objective -0.5148452
Found heuristic solution: objective -0.5531785

Root relaxation: objective -1.000000e+00, 580 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -1.00000    0  100   -0.5531

In [4]:
# inspect model
# m.write('model1.lp')
# !cat 'model1.lp'

### Optimizing 1-D Piecewise Linear Policies

Let $x \in [-1.0, 1.0]$ be a state variable and $a \in [-1.0, 1.0]$ be an action variable. We define $C(x, a)$ to be the following piecewise linear reward function:

$$
C(x, a) = \begin{cases}
|x + a + 1|, & x \leq 0 \\
|x + a - 1|, & x > 0 \\
\end{cases}.
$$

Let's consider a piecewise constant parameterized policy $\pi_{\theta}$ such that:
$$
\pi_{\theta}(x) = \begin{cases}
w_1 x + b_1, & l_1 \leq x \leq u_1 \\
w_2 x + b_2, & l_2 \leq x \leq u_2
\end{cases},
$$

where $\theta = \{ (w_i, b_i, l_i, u_i)~|~w_i, b_i, l_i, u_i \in \mathbb{R}, i=1,2 \}$ are the policy parameters.

Our objective is to minimize $C(x,a)$.

In [5]:
m = Model('PwL-Policy')

# generate states
N = 100
samples = [[x] for x in np.linspace(-0.90, 0.90, num=N)]
states = m.addVars(N, 1, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='x')
m.addConstrs((states[j, 0] == samples[j][0] for j in range(N)))

# encode PwL policy with rectlinear partition
K, D = 2, 1 
pi = PwLPolicy(m, D, K, action_lb=-1.0, action_ub=1.0, state_lb=-1.0, state_ub=1.0)
actions = []
for n in range(N):
    sample_n = samples[n]
    x_n = states.select(n, '*')
    a_n = pi(x_n, sample_n, n)
    actions.append(a_n)

# encode PwL cost function
for j in range(N):
    x_j, a_j = states.select(j, 0)[0], actions[j][0]
    c_j = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, obj=1.0/N, name='c{}'.format(j)) # minimize

    c1 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(c1 == x_j + a_j + 1)
    abs1 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addGenConstrAbs(abs1, c1)

    c2 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(c2 == x_j + a_j - 1)
    abs2 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addGenConstrAbs(abs2, c2)

    ic = m.addVar(vtype=GRB.BINARY, name='ic_x{}'.format(j))
    m.addGenConstrIndicator(ic, True,  x_j <= 0.0)
    m.addGenConstrIndicator(ic, False, x_j >= 0.0)
    m.addGenConstrIndicator(ic, True,  c_j == abs1)
    m.addGenConstrIndicator(ic, False, c_j == abs2)

# optimize policy parameters
m.optimize()

# report results
print('\n>> Policy:')
print(pi)

Optimize a model with 400 rows, 1508 columns and 800 nonzeros
Model has 100 SOS constraints
Model has 1900 general constraints
Variable types: 708 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-03, 1e+00]
Presolve added 1672 rows and 0 columns
Presolve removed 0 rows and 208 columns
Presolve time: 0.56s
Presolved: 2072 rows, 1300 columns, 5716 nonzeros
Presolved model has 200 SOS constraint(s)
Variable types: 608 continuous, 692 integer (692 binary)

Root relaxation: objective 6.418477e-17, 746 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  100          -    0.00000      -     -    0s
     0     0    0.00000    0  100          -    0.00000      -     -    0s
H    0     0                      

In [6]:
# inspect model
# m.write('model2.lp')
# !cat 'model2.lp'

### Optimizing 2-D Piecewise Linear Policies

Let $x_i \in [-1.0, 1.0]$ state variables and $a_i \in [-1.0, 1.0]$ be action variables for $i=1,2$. We define $C(x, a)$ to be the following piecewise linear reward function:

$$
C(x, a) = \begin{cases}
|x_1 + a_1| + |x_2 + a_2|, & x_1 \leq 0 \wedge x_2 \leq 0 \\
|x_1 + a_1 + 1| + |x_2 + a_2 - 1|, & x_1 \leq 0 \wedge x_2 > 0    \\
|x_1 + a_1 - 1| + |x_2 + a_2 + 1|, & x_1 > 0 \wedge x_2 \leq 0    \\
|x_1 + a_1| + |x_2 + a_2|, & x_1 > 0 \wedge x_2 > 0       \\
\end{cases}.
$$

Let's consider a piecewise constant parameterized policy $\pi_{\theta}$ such that:
$$
\pi_{\theta}(x) = \begin{cases}
(w_{11}^{\top}\cdot\tilde{x},~w_{12}^{\top}\cdot\tilde{x}), & l_{11} \leq x_1 \leq u_{11} \wedge l_{12} \leq x_2 \leq u_{12} \\
(w_{21}^{\top}\cdot\tilde{x},~w_{22}^{\top}\cdot\tilde{x}), & l_{11} \leq x_1 \leq u_{11} \wedge l_{12} \leq x_2 \leq u_{12} \\
(w_{31}^{\top}\cdot\tilde{x},~w_{32}^{\top}\cdot\tilde{x}), & l_{31} \leq x_1 \leq u_{31} \wedge l_{32} \leq x_2 \leq u_{32} \\
(w_{41}^{\top}\cdot\tilde{x},~w_{42}^{\top}\cdot\tilde{x}), & l_{41} \leq x_1 \leq u_{41} \wedge l_{42} \leq x_2 \leq u_{42} \\
\end{cases},
$$

where $\theta = \{ (w_{ij}, b_i, l_{ij}, u_{ij})~|~w_{ij}, b_i, l_{ij}, u_{ij} \in \mathbb{R}, i=1, \cdots, 4; j=1,2 \}$ are the policy parameters, and $\tilde{x} = [x, 1]^{\top}$.

Our objective is to minimize $C(x,a)$.

In [7]:
m = Model('PwL-Policy-2D')

# generate states
N = 10
D = 2
samples = list(itertools.product(*[np.linspace(-0.90, 0.90, num=N)] * D)) # 10 x 10 == 100 states
states = m.addVars(N**D, D, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='x')
m.addConstrs((states[n, i] == samples[n][i] for n in range(N**D) for i in range(D)))

# encode PwL policy with rectlinear partition
K = 4
pi = PwLPolicy(m, D, K, action_lb=-1.0, action_ub=1.0, state_lb=-1.0, state_ub=1.0)

# encode PwL cost function
c = m.addVars(N**D, lb=-GRB.INFINITY, ub=GRB.INFINITY, obj=1.0/(N**D), name='c') # minimize

for n in range(N**D):
    x = states.select(n, '*')
    sample = samples[n][:]
    a = pi(x, sample, n)
    
    case = m.addVars(4, vtype=GRB.BINARY)
    bounds = m.addVars(4, 2, vtype=GRB.BINARY)

    m.addConstrs((bounds[0, d] == True)  >> (x[d] <= 0) for d in range(D))
    m.addConstrs((bounds[0, d] == False) >> (x[d] >= 0) for d in range(D))

    m.addConstr((bounds[1, 0] == True)  >> (x[0] <= 0))
    m.addConstr((bounds[1, 0] == False) >> (x[0] >= 0))
    m.addConstr((bounds[1, 1] == True)  >> (x[1] >= 0))
    m.addConstr((bounds[1, 1] == False) >> (x[1] <= 0))
    
    m.addConstr((bounds[2, 0] == True)  >> (x[0] >= 0))
    m.addConstr((bounds[2, 0] == False) >> (x[0] <= 0))
    m.addConstr((bounds[2, 1] == True)  >> (x[1] <= 0))
    m.addConstr((bounds[2, 1] == False) >> (x[1] >= 0))
    
    m.addConstrs((bounds[3, d] == True)  >> (x[d] >= 0) for d in range(D))
    m.addConstrs((bounds[3, d] == False) >> (x[d] <= 0) for d in range(D))
    
    m.addConstrs((case[k] == and_(*bounds.select(k, '*'))) for k in range(K))
    
    m.addSOS(GRB.SOS_TYPE1, [*case.select('*')])
    partition = m.addVar(vtype=GRB.BINARY)
    m.addConstr(partition == or_(*case.select('*')))
    m.addConstr(partition == True)

    # |x_1 + a_1| + |x_2 + a_2|
    aux1 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux1 == x[0] + a[0])
    aux2 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux2 == abs_(aux1))
    aux3 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux3 == x[1] + a[1])
    aux4 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux4 == abs_(aux3))
    m.addConstr((case[0] == True) >> (c[n] == aux2 + aux4))
    
    # |x_1 + a_1 + 1| + |x_2 + a_2 - 1|
    aux1 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux1 == x[0] + a[0] + 1)
    aux2 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux2 == abs_(aux1))
    aux3 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux3 == x[1] + a[1] - 1)
    aux4 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux4 == abs_(aux3))
    m.addConstr((case[1] == True) >> (c[n] == aux2 + aux4))
    
    # |x_1 + a_1 - 1| + |x_2 + a_2 + 1|
    aux1 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux1 == x[0] + a[0] - 1)
    aux2 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux2 == abs_(aux1))
    aux3 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux3 == x[1] + a[1] + 1)
    aux4 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux4 == abs_(aux3))
    m.addConstr((case[2] == True) >> (c[n] == aux2 + aux4))

    # |x_1 + a_1| + |x_2 + a_2|
    aux1 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux1 == x[0] + a[0])
    aux2 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux2 == abs_(aux1))
    aux3 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY)
    m.addConstr(aux3 == x[1] + a[1])
    aux4 = m.addVar(lb=0.0, ub=GRB.INFINITY)
    m.addConstr(aux4 == abs_(aux3))
    m.addConstr((case[3] == True) >> (c[n] == aux2 + aux4))
    
# optimize policy parameters
m.optimize()

# report results
print('\n>> Policy:')
print(pi)

Optimize a model with 1200 rows, 5540 columns and 2800 nonzeros
Model has 200 SOS constraints
Model has 7800 general constraints
Variable types: 2140 continuous, 3400 integer (3400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
Presolve added 6132 rows and 0 columns
Presolve removed 0 rows and 452 columns
Presolve time: 0.78s
Presolved: 7332 rows, 5088 columns, 21272 nonzeros
Presolved model has 800 SOS constraint(s)
Variable types: 2824 continuous, 2264 integer (2248 binary)

Root relaxation: objective 0.000000e+00, 1795 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  200          -    0.00000      -     -    1s
     0     0    0.00000    0  200          -    0.00000      -     -    1s
     0     2    0.00000 

In [8]:
# inspect model
# m.write('model3.lp')
# !cat 'model3.lp'